# Two goals:
### 1. collect candidates' profile info
### 2. filter based on skillsets


In [1]:
import pandas as pd
import os
import PyPDF2
import docx 
from shutil import copy2

# from os import listdir
# from os.path import isfile, join
# from io import StringIO
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()  ## nlp model for phrase match
# from spacy.matcher import PhraseMatcher


In [ ]:
# !conda install -c conda-forge python-docx 

In [2]:
def pdfextract(file):
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    countpage = fileReader.getNumPages()
    count = 0
    text = []
    while count < countpage:    
        pageObj = fileReader.getPage(count)
        count +=1
        t = pageObj.extractText()
        text.append(t)
    return text

def docextract(file):
    doc = docx.Document(file)
    fullText = []
    for paragraph in doc.paragraphs:
        fullText.append(paragraph.text)
    return fullText

def clean_pdf(text):
    text = str(text)
    return text.lower().replace("\\n", "")

def clean_word(text):
    text = str(' '.join(p for p in text))
    return text.lower().replace("\\n", "")


def extract_all(resume_path):
    file_name = []
    file_text = []

    for f in os.listdir(resume_path):
        if os.path.isfile(os.path.join(resume_path, f)):
            if '.pdf' in f:
                file_text.append(clean_pdf(pdfextract(os.path.join(resume_path, f))))
                file_name.append(f)
            elif '.docx' in f:
                file_text.append(clean_word(docextract(os.path.join(resume_path, f))))
                file_name.append(f)
            else:
                print(f+' is not pdf or docx format, so it is not readable')
    return pd.DataFrame({'file_names': file_name, 'contents': file_text})
    

def search(df_all_resumes, keywords):
    search_result=[]
    contained_keywords=[]
    
    and_keywords = keywords[0]
    or_keywords = keywords[1]
    
    and_length=len(and_keywords)
    or_length= len(or_keywords)


    for i in range(len(df_all_resumes.file_names)):
        contents = df_all_resumes.contents[i]
        file_name = df_all_resumes.file_names[i]
        
        ##check and_keywords
        if_contain =False
        if and_keywords!=[]:
            for k in and_keywords:
                if_contain += (k in contents)

        
        ##check or_keywords
        check= 0
        contain=[]
        if or_keywords!=[]:
            for word in or_keywords:
                if word in contents:
                    contain.append(word)
                    check+=1

#    & if_contain == and_length      
        if (and_length!=0) & (or_length!=0):
            if (check>0) & (if_contain==and_length):
                contain+=and_keywords
                contained_keywords.append(contain)
                search_result.append(file_name)
        elif (and_length==0) & (or_length!=0):
            if check>0:
                contained_keywords.append(contain)
                search_result.append(file_name)
        elif (and_length!=0) & (or_length==0):
            if if_contain == and_length:
                contained_keywords.append(and_keywords)
                search_result.append(file_name)
        else:
            search_result.append(file_name)
#     print(len(search_result))
#     print(len(contained_keywords))
    df = pd.DataFrame({'file_names':search_result, 'contained_keywords': contained_keywords})
    print(str(len(df.file_names)) + ' resumes found')
    return df


def see_resume(result, current_index):
    name = result.file_names[current_index]
    print('This resume '+ name + " contains "+ str(result.contained_keywords[current_index]))
    print('Current index is ' + str(current_index))    
    os.startfile(os.path.join(resume_path, name))
    return name
    
    

## Customize your search here

In [3]:
resume_path='sample resume bundle'
and_keywords = ['master']
or_keywords = ['python', 'r']
result = search(extract_all(resume_path), [and_keywords, or_keywords])

2 resumes found


In [4]:
result

,file_names,contained_keywords
0,Nagadhatri_Chennavajula.docx,"[python, r, master]"
1,priya_sai (1).docx,"[python, r, master]"


## Read the searched resume here

In [9]:
current_index = 1   ### Only run it at the first time

In [10]:
name = see_resume(result, current_index)
current_index+=1

This resume priya_sai (1).docx contains ['python', 'r', 'master']
Current index is 1


## Save this resume in a new folder

In [11]:
result_path = 'test'
copy2(os.path.join(resume_path, name), os.path.join(result_path, name))

'test\\priya_sai (1).docx'